In [400]:
import requests
import pandas as pd
import pprint as pprint
import json
from itertools import izip
from pymongo import MongoClient
from bson.objectid import ObjectId

In [447]:
cd ../san_fran/

/Users/michaelhood/Documents/projects/project/data/san_fran


In [448]:
df = pd.read_csv('trx_sf.csv').drop(['Unnamed: 0'], axis=1)

In [452]:
df.info().T

<class 'pandas.core.frame.DataFrame'>
Int64Index: 764 entries, 0 to 763
Data columns (total 45 columns):
_id                           764 non-null object
city                          764 non-null object
latitude                      764 non-null float64
longitude                     764 non-null float64
state                         764 non-null object
street                        764 non-null object
zipcode                       764 non-null int64
appliances                    380 non-null object
architecture                  342 non-null object
basement                      226 non-null object
bathrooms                     747 non-null float64
bedrooms                      724 non-null float64
coolingsystem                 125 non-null object
coveredparkingspaces          205 non-null float64
exteriormaterial              267 non-null object
finishedsqft                  741 non-null float64
floorcovering                 365 non-null object
floornumber                   69 non-nul

AttributeError: 'NoneType' object has no attribute 'T'

In [453]:
def query_api(df):
    
    WSAPI_KEY = "e54c50263609ee055aece36205c4827c"
    TRANSIT_BASE_URL = "http://transit.walkscore.com/transit/score/?lat={0}&lon={1}&city={2}&state={3}&wsapikey={4}&research=yes"
    WALK_BASE_URL = "http://api.walkscore.com/score?format={0}&address={1}%20{2}%20{3}%20{4}&lat={5}&lon={6}&wsapikey={7}"
    
    walk_scores = []
    transit_scores = []
    
    df['city'] = df['city'].apply(lambda city: city.replace(' ', '-'))
    df['city'] = df['city'].apply(lambda city: "Seattle" if city=="Des-Moines" else city) 
    df['street'] = df['street'].apply(lambda street: street[:street.find('#')-1] if '#' in street else street)
    df['street'] = df['street'].apply(lambda street: street.replace(' ', '-'))
    
    count = 1
    for row in df.iterrows():
        city = row[1][1]
        lat = row[1][2]
        lon = row[1][3]
        state = row[1][4]
        address = row[1][5]
        zip_code = row[1][6]
        walk_api_query = WALK_BASE_URL.format('json', address, zip_code, city, state, lat, lon, WSAPI_KEY) 
        walk_api_response = requests.get(walk_api_query)
        walk_scores.append(walk_api_response.content)
        transit_api_query = TRANSIT_BASE_URL.format(lat, lon, city, state, WSAPI_KEY)
        transit_api_response = requests.get(transit_api_query)
        transit_scores.append(transit_api_response.content)
        print "Querying row: {0}".format(count)
        count += 1
    return walk_scores, transit_scores

In [454]:
walk_scores, transit_scores = query_api(df)

Querying row: 1
Querying row: 2
Querying row: 3
Querying row: 4
Querying row: 5
Querying row: 6
Querying row: 7
Querying row: 8
Querying row: 9
Querying row: 10
Querying row: 11
Querying row: 12
Querying row: 13
Querying row: 14
Querying row: 15
Querying row: 16
Querying row: 17
Querying row: 18
Querying row: 19
Querying row: 20
Querying row: 21
Querying row: 22
Querying row: 23
Querying row: 24
Querying row: 25
Querying row: 26
Querying row: 27
Querying row: 28
Querying row: 29
Querying row: 30
Querying row: 31
Querying row: 32
Querying row: 33
Querying row: 34
Querying row: 35
Querying row: 36
Querying row: 37
Querying row: 38
Querying row: 39
Querying row: 40
Querying row: 41
Querying row: 42
Querying row: 43
Querying row: 44
Querying row: 45
Querying row: 46
Querying row: 47
Querying row: 48
Querying row: 49
Querying row: 50
Querying row: 51
Querying row: 52
Querying row: 53
Querying row: 54
Querying row: 55
Querying row: 56
Querying row: 57
Querying row: 58
Querying row: 59
Queryi

In [270]:
def score_api_responses(walk_scores, transit_scores):
    response_status = {}
    walk_succeed_string = '"status": 1' 
    transit_fail_string = "You must provide a valid 'city' and 'state'"
    
    for index, score in enumerate(izip(walk_scores, transit_scores)):
        status = {'walkscore_status': 1,
                  'transitscore_status': 1} # 1 --> status is good
        if walk_succeed_string not in score[0]:
            status['transitscore_status'] = 0 # status if bad
        if transit_fail_string in score[1]:
            status['transitscore_status'] = 0
            #response_status[index] = 400
        #else:
            #response_status[index] = 0
        response_status[index] = status
    return response_status
            

In [359]:
def get_api_report(response_status):
    success_count = 0
    incomplete_count = 0
    incompletes = []
    
    for key, value in response_status.items():
        if (value['transitscore_status'] == 0) or (value['walkscore_status'] == 0):
            incomplete_count += 1
            incompletes.append({key: value})
        else:
            success_count += 1
    print "{0} API calls were successful".format(success_count)
    print "{0} API calls were incomplete or unsuccessful".format(incomplete_count)
    return incompletes            

In [455]:
response_status = check_api_responses(walk_scores, transit_scores)

In [460]:
incompletes = get_api_report(response_status)

764 API calls were successful
0 API calls were incomplete or unsuccessful


In [130]:
def to_json(query_response):
    return json.loads(query_response.replace('\n', ''))

In [461]:
def get_and_format_data(df, walk_scores, transit_scores):
    
    key_ids = []
    walkscore_descriptions = []
    walkscore_scores = []
    trans_descriptions = []
    trans_summaries = []
    trans_scores = []
    
    for i in range(min([len(walk_scores), len(transit_scores)])): # take the smaller of the two lists
        key_id = df.ix[i][0]
        key_ids.append(key_id)
        try:
            walkscore_desc = to_json(walk_scores[i])['description']
        except:
            walkscore_desc = 'None'
        walkscore_descriptions.append(walkscore_desc)
        try:
            walkscore_score = to_json(walk_scores[i])['walkscore']
        except:
            walkscore_score = 'None'
        walkscore_scores.append(walkscore_score)
        try:
            trans_desc = to_json(transit_scores[i])['description']
        except:
            trans_desc = 'None'
        trans_descriptions.append(trans_desc)
        try:
            trans_summ = to_json(transit_scores[i])['summary']
        except:
            trans_summ = 'None'
        trans_summaries.append(trans_summ)
        try:
            trans_score = to_json(transit_scores[i])['transit_score']
        except:
            trans_score = "None"
        trans_scores.append(trans_score)
    
    return pd.DataFrame({'_id': key_ids, 
                         'walkscore_desc': walkscore_descriptions, 
                         'walkscore_score': walkscore_scores,
                         'trans_desc': trans_desc,
                         'trans_summary': trans_summaries,
                         'trans_score': trans_scores}, index=range(len(key_ids))) 

In [462]:
df_api = get_and_format_data(df, walk_scores, transit_scores)

In [463]:
#df_api

In [363]:
client = MongoClient()
db = client.proj

In [464]:
df_new = pd.concat([df, df_api], axis=1, join='outer')

In [466]:
df_new.shape

(764, 51)

In [443]:
def update_db(df):
    for row in df.iterrows():
        mongo_id = ObjectId(row[1][0])
        trans_desc = row[1][44]
        trans_score = row[1][45]
        trans_summary = row[1][46]
        walkscore_desc = row[1][47]
        walkscore_score = row[1][48]
        print "updating...    ", mongo_id
        db.listings.update(
                            {'_id': mongo_id},
                            { '$set': { "trans_desc": trans_desc,
                                        "trans_score": trans_score,
                                        "trans_summary": trans_summary,
                                        "walkscore_desc": walkscore_desc,
                                        "walkscore_score": walkscore_score
                            }}
                            )

In [467]:
update_db(df_new)

updating...     55f6e8a63ae7404c5edef1e6
updating...     55f6e8a63ae7404c5edef1e7
updating...     55f6e8a63ae7404c5edef1e8
updating...     55f6e8a63ae7404c5edef1e9
updating...     55f6e8a63ae7404c5edef1ea
updating...     55f6e8a63ae7404c5edef1eb
updating...     55f6e8a63ae7404c5edef1ec
updating...     55f6e8a63ae7404c5edef1ed
updating...     55f6e8a63ae7404c5edef1ee
updating...     55f6e8a63ae7404c5edef1ef
updating...     55f6e8a63ae7404c5edef1f0
updating...     55f6e8a63ae7404c5edef1f1
updating...     55f6e8a63ae7404c5edef1f2
updating...     55f6e8a63ae7404c5edef1f3
updating...     55f6e8a63ae7404c5edef1f4
updating...     55f6e8a63ae7404c5edef1f5
updating...     55f6e8a63ae7404c5edef1f6
updating...     55f6e8a63ae7404c5edef1f7
updating...     55f6e8a63ae7404c5edef1f8
updating...     55f6e8a63ae7404c5edef1f9
updating...     55f6e8a63ae7404c5edef1fa
updating...     55f6e8a63ae7404c5edef1fb
updating...     55f6e8a63ae7404c5edef1fc
updating...     55f6e8a63ae7404c5edef1fd
updating...     